<a href="https://colab.research.google.com/github/ikasuryani/Sanbercode0920FinalProject/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd 
trainlink = 'https://raw.githubusercontent.com/ikasuryani/Sanbercode0920FinalProject/main/train.csv'
testlink = 'https://raw.githubusercontent.com/ikasuryani/Sanbercode0920FinalProject/main/test.csv'

df = pd.read_csv(trainlink)

#Encoding
df = pd.get_dummies(df, columns=['Jenis Kelamin', 'Status Perkawinan'])
df = pd.get_dummies(df, columns=['Gaji'], drop_first=True)
df = df.rename(columns={'Gaji_>7jt':'Gaji'})


df.head()

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Pekerjaan,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Jenis Kelamin_Laki2,Jenis Kelamin_Perempuan,Status Perkawinan_Belum Pernah Menikah,Status Perkawinan_Berpisah,Status Perkawinan_Cerai,Status Perkawinan_Janda,Status Perkawinan_Menikah,Status Perkawinan_Menikah LDR,Gaji
0,0,21,Wiraswasta,242912,SMA,9,Servis Lainnya,0.0,0.0,35.0,0,1,1,0,0,0,0,0,0
1,1,49,Wiraswasta,140782,10th,6,Ekesekutif Managerial,0.0,0.0,40.0,0,1,0,0,1,0,0,0,0
2,2,44,Wiraswasta,120057,D3,12,Ekesekutif Managerial,61404000.0,0.0,45.0,1,0,0,0,0,0,1,0,1
3,3,24,Wiraswasta,194630,Sarjana,13,Spesialis,0.0,0.0,35.0,1,0,1,0,0,0,0,0,0
4,4,33,Wiraswasta,219619,Master,14,Spesialis,210336000.0,0.0,40.0,1,0,0,0,0,0,1,0,1


In [36]:
df['Pendidikan'].value_counts()
# mask = df['Pendidikan'].isnull()
# print(df[mask])

SMA                     11666
Pendidikan Tinggi        8005
Sarjana                  5928
Master                   1944
D4                       1500
11th                     1315
D3                       1175
10th                     1024
7th-8th                   682
Sekolah Professional      622
9th                       570
12th                      478
Doktor                    451
5th-6th                   388
1st-4th                   184
SD                         62
Name: Pendidikan, dtype: int64

In [12]:
#Fill Missing Value
from sklearn.impute import KNNImputer

imp = KNNImputer(n_neighbors=3)
imp.fit_transform(df)

ValueError: ignored